In [1]:
import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision.models import ResNet18_Weights

In [2]:
# 1. Проверка устройства (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# 2. Определение трансформаций для данных
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [5]:
# 3. Загрузка и разделение датасета
dataset = torchvision.datasets.ImageFolder(
    root='../images',
    transform=train_transform
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [6]:
# 4. Создание DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# 5. Инициализация модели
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Замораживаем все слои, кроме последнего
for param in model.parameters():
    param.requires_grad = False

# Заменяем последний слой для нашей задачи (2 класса)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(device)

In [8]:
# 6. Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([
    {'params': model.fc.parameters(), 'lr': 0.001}
], lr=0.0001)

scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [9]:
# 7. Цикл обучения
num_epochs = 15
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Валидация
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # Обновление learning rate
    scheduler.step()
    
    current_lr = optimizer.param_groups[0]['lr']
    val_accuracy = 100 * correct / total
    
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Learning Rate: {current_lr:.6f}")
    print(f"Train Loss: {running_loss/len(train_loader):.4f}")
    print(f"Val Loss: {val_loss/len(val_loader):.4f}")
    print(f"Val Accuracy: {val_accuracy:.2f}%")
    
    # Сохранение лучшей модели
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save({
            'model_state_dict': model.state_dict(),
            'class_to_idx': dataset.class_to_idx,
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'best_accuracy': best_accuracy
        }, 'best_tire_classifier.pth')
        print("New best model saved!")


Epoch 1/15
Learning Rate: 0.001000
Train Loss: 0.5011
Val Loss: 0.4448
Val Accuracy: 76.61%
New best model saved!

Epoch 2/15
Learning Rate: 0.001000
Train Loss: 0.3441
Val Loss: 0.3426
Val Accuracy: 84.14%
New best model saved!

Epoch 3/15
Learning Rate: 0.001000
Train Loss: 0.3060
Val Loss: 0.3083
Val Accuracy: 87.10%
New best model saved!

Epoch 4/15
Learning Rate: 0.001000
Train Loss: 0.3096
Val Loss: 0.3405
Val Accuracy: 84.95%

Epoch 5/15
Learning Rate: 0.000100
Train Loss: 0.2752
Val Loss: 0.2876
Val Accuracy: 88.98%
New best model saved!

Epoch 6/15
Learning Rate: 0.000100
Train Loss: 0.2704
Val Loss: 0.2622
Val Accuracy: 90.59%
New best model saved!

Epoch 7/15
Learning Rate: 0.000100
Train Loss: 0.2762
Val Loss: 0.2821
Val Accuracy: 87.37%

Epoch 8/15
Learning Rate: 0.000100
Train Loss: 0.2646
Val Loss: 0.2681
Val Accuracy: 89.25%

Epoch 9/15
Learning Rate: 0.000100
Train Loss: 0.2622
Val Loss: 0.2814
Val Accuracy: 87.63%

Epoch 10/15
Learning Rate: 0.000010
Train Loss: 0.25

In [ ]:
# 8. Загрузка лучшей модели для использования
checkpoint = torch.load('best_tire_classifier.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"\nTraining completed! Best validation accuracy: {best_accuracy:.2f}%")